In [1]:
# 필요한 라이브러리 설치
!pip install accelerate
!pip install diffusers transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 4.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [3]:
#!pip install facenet-pytorch
!pip install torch torchvision

In [2]:
!pip install face_recognition

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.1/100.1 MB 9.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566170 sha256=5f69596ebd005076444ba7bdb290aae387d7e5b06ddd84b278d514e0eafa2c44
  Stored in directory: /root/.cache/pip/wheels/7a/eb/cf/e9eced74122b679557f597bb7c8e4c739cfcac526db1fd523d
Successfully built face-recognition-models


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
import os
import random
# 이미지가 있는 디렉토리 경로 설정
directory_path = '/content/drive/MyDrive/ArtificialIntelligence/Project/train/real'

# 디렉토리 내의 모든 파일 목록 가져오기
all_files = []
for f in os.listdir(directory_path):
  full_path = os.path.join(directory_path, f)
  if os.path.isfile(full_path):
    all_files.append(full_path)

# 파일 목록에서 무작위로 선택
random.seed(42)

if len(all_files) > 10:
    selected_files = random.sample(all_files, 10)
else:
    selected_files = all_files

In [7]:
import numpy as np
import torch
from PIL import Image
from torchvision import transforms
from torchvision.models.segmentation import deeplabv3_resnet50
from diffusers import StableDiffusionImg2ImgPipeline
from transformers import pipeline

/usr/local/lib/python3.10/dist-packages/diffusers/models/transformers/transformer_2d.py:34: FutureWarning: `Transformer2DModelOutput` is deprecated and will be removed in version 1.0.0. Importing `Transformer2DModelOutput` from `diffusers.models.transformer_2d` is deprecated and this will be removed in a future version. Please use `from diffusers.models.modeling_outputs import Transformer2DModelOutput`, instead.
  deprecate("Transformer2DModelOutput", "1.0.0", deprecation_message)


In [8]:
def load_segmentation_model():
    model = deeplabv3_resnet50(pretrained=True)
    model.eval()
    return model

def segment_face(image):
    preprocess = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        transforms.Resize((512, 512))
    ])

    input_tensor = preprocess(image)
    input_batch = input_tensor.unsqueeze(0)

    with torch.no_grad():
        model = load_segmentation_model()
        output = model(input_batch)['out'][0]
        output_predictions = output.argmax(0)

    # 가정: 클래스 15가 얼굴과 관련된 부위
    face_mask = (output_predictions == 15).to(torch.uint8)
    return face_mask

def modify_face(image_path, prompt):
    image = Image.open(image_path) #.convert('RGB')
    face_mask = segment_face(image)

    # 세그먼테이션 마스크 적용
    image_np = torch.tensor(np.array(image))
    face_image = Image.fromarray((image_np * face_mask.unsqueeze(-1)).numpy().astype(np.uint8))

    # Diffusion 모델 로드
    pipe = StableDiffusionImg2ImgPipeline.from_pretrained("CompVis/stable-diffusion-v1-4", torch_dtype=torch.float16)
    pipe = pipe.to("cuda")

    processed_face_image = transforms.ToTensor()(face_image).unsqueeze(0).to("cuda")
    modified_face = pipe(prompt=prompt, image=processed_face_image, num_inference_steps=50, strength=0.75).images[0]

    return modified_face

# 사용 예
image_path = "/content/drive/MyDrive/ArtificialIntelligence/Project/train/real/22480.jpg"
prompt = "An elderly Asian woman's face, realistic style"
modified_image = modify_face(image_path, prompt)
modified_image.show()


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DeepLabV3_ResNet50_Weights.COCO_WITH_VOC_LABELS_V1`. You can also use `weights=DeepLabV3_ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/deeplabv3_resnet50_coco-cd0a2569.pth" to /root/.cache/torch/hub/checkpoints/deeplabv3_resnet50_coco-cd0a2569.pth
100%|██████████| 161M/161M [00:01<00:00, 160MB/s]


RuntimeError: The size of tensor a (256) must match the size of tensor b (512) at non-singleton dimension 1

In [ ]:
from PIL import Image
import torch
import numpy as np
from torchvision import transforms
from torchvision.models.segmentation import deeplabv3_resnet50
from diffusers import StableDiffusionImg2ImgPipeline
from transformers import pipeline

def load_segmentation_model():
    model = deeplabv3_resnet50(pretrained=True)
    model.eval()
    return model

def segment_face(image):
    # 원본 이미지 크기 저장
    orig_size = image.size
    preprocess = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        transforms.Resize((512, 512))
    ])
    input_tensor = preprocess(image)
    input_batch = input_tensor.unsqueeze(0)

    with torch.no_grad():
        model = load_segmentation_model()
        output = model(input_batch)['out'][0]
        output_predictions = output.argmax(0)

    # 세그먼테이션 마스크를 원본 이미지 크기로 조정
    resize = transforms.Resize(orig_size, interpolation=Image.NEAREST)
    face_mask = resize(output_predictions == 15).to(torch.uint8)
    return face_mask

def modify_face(image_path, prompt):
    image = Image.open(image_path).convert('RGB')
    face_mask = segment_face(image)

    # 세그먼테이션 마스크 적용
    image_np = np.array(image)
    face_image_np = (image_np * np.array(face_mask).reshape(*face_mask.shape, 1)).astype(np.uint8)
    face_image = Image.fromarray(face_image_np)

    # Diffusion 모델 로드
    pipe = StableDiffusionImg2ImgPipeline.from_pretrained("CompVis/stable-diffusion-v1-4", torch_dtype=torch.float16)
    pipe = pipe.to("cuda")

    processed_face_image = transforms.ToTensor()(face_image).unsqueeze(0).to("cuda")
    modified_face = pipe(prompt=prompt, init_image=processed_face_image, num_inference_steps=50, strength=0.75).images[0]

    return modified_face

# 사용 예
image_path = "/content/drive/MyDrive/ArtificialIntelligence/Project/train/real/22480.jpg"
prompt = "An elderly Asian woman's face, realistic style"
modified_image = modify_face(image_path, prompt)
modified_image.show()


ValueError: Input and output must have the same number of spatial dimensions, but got input with spatial dimensions of [512] and output size of [256, 256]. Please provide input tensor in (N, C, d1, d2, ...,dK) format and output size in (o1, o2, ...,oK) format.

In [ ]:
import torch
from PIL import Image
import numpy as np
import face_recognition
from torchvision import transforms
from torchvision.models.segmentation import deeplabv3_resnet50
from diffusers import StableDiffusionImg2ImgPipeline
from transformers import pipeline

def load_segmentation_model():
    model = deeplabv3_resnet50(pretrained=True)
    model.eval()
    return model

def segment_face(image, model):
    # 원본 이미지 크기 저장
    orig_size = image.size

    # 이미지 전처리
    preprocess = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        transforms.Resize((512, 512))
    ])

    # 입력 텐서 준비
    input_tensor = preprocess(image).unsqueeze(0)  # 배치 차원 추가

    with torch.no_grad():
        output = model(input_tensor)['out'][0]
        output_predictions = output.argmax(0)

    # 출력 텐서를 원본 이미지 크기로 조정
    face_mask = transforms.functional.resize(output_predictions.unsqueeze(0).float(), orig_size, interpolation=transforms.InterpolationMode.NEAREST).byte().squeeze()

    return face_mask

    # 세그먼테이션 마스크를 원본 이미지 크기로 조정
    resize = transforms.Resize(image.size, interpolation=Image.NEAREST)
    face_mask = resize(output_predictions == 15).to(torch.uint8)
    return face_mask

def modify_face(image_path, prompt):
    device = "cuda" if torch.cuda.is_available() else "cpu"
    pipe = StableDiffusionImg2ImgPipeline.from_pretrained("CompVis/stable-diffusion-v1-4", torch_dtype=torch.float16, safety_checker = None).to(device)

    image = Image.open(image_path).convert('RGB')
    model = load_segmentation_model()
    face_mask = segment_face(image, model)

    masked_image = np.array(image) * np.array(face_mask).reshape(*face_mask.shape, 1)
    face_image = Image.fromarray(masked_image)

    # Diffusion 모델을 이용한 얼굴 변형
    processed_face_image = transforms.ToTensor()(face_image).unsqueeze(0).to(device)
    modified_face = pipe(prompt=prompt, image=processed_face_image, num_inference_steps=50, strength=0.75).images[0]

    # 변형된 얼굴을 원본 이미지에 다시 합성
    final_image = Image.fromarray(np.array(image))
    final_image.paste(modified_face.convert('RGB'), mask=Image.fromarray((face_mask.numpy() * 255).astype(np.uint8)))
    final_path = "/content/drive/MyDrive/ArtificialIntelligence/Project/GeneratedImages/image.jpg"
    final_image.save(final_path)

    return final_image

# 사용 예
image_path = "/content/drive/MyDrive/ArtificialIntelligence/Project/train/real/22480.jpg"
prompt = "A young Asian female face, realistic style"

modified_image = modify_face(image_path, prompt)
#modified_image.show()


Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

You have disabled the safety checker for <class 'diffusers.pipelines.stable_diffusion.pipeline_stable_diffusion_img2img.StableDiffusionImg2ImgPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve analyzing network behavior or auditing its results. For more information, please have a look at https://github.com/huggingface/diffusers/pull/254 .
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a wei

  0%|          | 0/37 [00:00<?, ?it/s]

In [ ]:
import torch
from PIL import Image
import numpy as np
from torchvision import transforms
from diffusers import StableDiffusionImg2ImgPipeline
from transformers import pipeline

def load_segmentation_model():
    model = torch.hub.load('pytorch/vision:v0.10.0', 'deeplabv3_resnet101', pretrained=True)
    model.eval()
    return model

def segment_face(image, model):
    preprocess = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        transforms.Resize((512, 512))
    ])
    input_tensor = preprocess(image).unsqueeze(0)

    with torch.no_grad():
        output = model(input_tensor)['out'][0]
    output_predictions = output.argmax(0)
    mask = output_predictions == 15  # 얼굴 및 관련 부위 가정
    return mask

def modify_face(image_path, prompt, model):
    device = "cuda" if torch.cuda.is_available() else "cpu"
    image = Image.open(image_path).convert('RGB')
    mask = segment_face(image, model).float()

    # 세그먼테이션 마스크 적용
    masked_image = np.array(image) * np.tile(mask.numpy()[..., None], (1, 1, 3))
    face_image = Image.fromarray(masked_image.astype(np.uint8))

    # Diffusion 모델 초기화
    pipe = StableDiffusionImg2ImgPipeline.from_pretrained("CompVis/stable-diffusion-v1-4", torch_dtype=torch.float16, revision="fp16").to(device)
    processed_face_image = transforms.ToTensor()(face_image).unsqueeze(0).to(device)

    # 이미지 변형 실행
    modified_image = pipe(prompt=prompt, init_image=processed_face_image, num_inference_steps=75, strength=0.75).images[0]

    # 결과 이미지 리턴
    final_image = Image.fromarray(np.array(image))
    final_image.paste(modified_image.convert('RGB'), mask=Image.fromarray((mask.numpy() * 255).astype(np.uint8)))
    final_path = "/content/drive/MyDrive/ArtificialIntelligence/Project/GeneratedImages/image.jpg"
    final_image.save(final_path)

    return final_image

# 모델 로드
model = load_segmentation_model()

# 사용 예
image_path = "/content/drive/MyDrive/ArtificialIntelligence/Project/train/real/22480.jpg"
prompt = "A young Asian female face, realistic style"
modified_image = modify_face(image_path, prompt, model)
modified_image.show()

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0


ValueError: operands could not be broadcast together with shapes (256,256,3) (512,512,3) 

In [12]:
def segment_face(image, model):
    original_size = image.size  # 원본 이미지 크기 저장
    preprocess = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        transforms.Resize((512, 512))  # 모델에 맞게 이미지 크기 조정
    ])
    input_tensor = preprocess(image).unsqueeze(0)

    with torch.no_grad():
        output = model(input_tensor)['out'][0]
    output_predictions = output.argmax(0)

    # 마스크를 원본 이미지 크기로 조정
    resize = transforms.Resize(original_size, interpolation=transforms.InterpolationMode.NEAREST)
    mask = resize(output_predictions.unsqueeze(0)).squeeze(0) == 15
    return mask.float()  # 마스크를 float 타입으로 변환

def modify_face(image_path, prompt, model):
    device = "cuda" if torch.cuda.is_available() else "cpu"
    image = Image.open(image_path).convert('RGB')
    mask = segment_face(image, model)

    # 세그먼테이션 마스크 적용
    masked_image = np.array(image) * np.tile(mask.numpy()[..., None], (1, 1, 3))
    face_image = Image.fromarray(masked_image.astype(np.uint8))

    # Diffusion 모델 초기화
    pipe = StableDiffusionImg2ImgPipeline.from_pretrained("CompVis/stable-diffusion-v1-4", torch_dtype=torch.float16, safety_checker = None).to(device)
    processed_face_image = transforms.ToTensor()(face_image).unsqueeze(0).to(device)

    # 이미지 변형 실행
    modified_image = pipe(prompt=prompt, image=processed_face_image, num_inference_steps=75, strength=0.75).images[0]

    # 결과 이미지 리턴
    final_image = Image.fromarray(np.array(image))
    final_image.paste(modified_image.convert('RGB'), mask=Image.fromarray((mask.numpy() * 255).astype(np.uint8)))
    final_path = "/content/drive/MyDrive/ArtificialIntelligence/Project/GeneratedImages/image.jpg"
    final_image.save(final_path)

    return final_image

# 모델 로드
model = load_segmentation_model()

# 사용 예
image_path = "/content/drive/MyDrive/ArtificialIntelligence/Project/train/real/22480.jpg"
prompt = "A young Asian female face, realistic style"
modified_image = modify_face(image_path, prompt, model)
modified_image.show()


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DeepLabV3_ResNet50_Weights.COCO_WITH_VOC_LABELS_V1`. You can also use `weights=DeepLabV3_ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

You have disabled the safety checker for <class 'diffusers.pipelines.stable_diffusion.pipeline_stable_diffusion_img2img.StableDiffusionImg2ImgPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve analyzing network behavior or auditing its results. For more information, please have a look at https://github.com/huggingface/diffusers/pull/254 .


  0%|          | 0/56 [00:00<?, ?it/s]